In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import random
import os.path as op
import scipy
import sklearn

from model.net import Net
from utils import NetworkData

In [2]:
learning_rate = 0.02
weight_decay = 5e-4
num_epochs = 10
# val_ratio = 0.2 

In [3]:
data = NetworkData(cache=None, save=False)
adj, L, x_user, x_item, x_train_edge, x_test_edge, y_train = data.get_data()
data.describe()

34404 419840
building adjacency ...
normalize adjacency ...
building node features ...
building node features ...
building edge features ...
building edge features ...
get data: 	number of users: 34404
number of songs: 419840
adjacency: (454244, 454244)
Lap: (454244, 454244)
dim of user feature: 6


In [4]:
# norm
x_user = x_user / x_user.sum(1, keepdims=True)
x_item = x_item / x_item.sum(1, keepdims=True)

/var/folders/xh/jth0z5cd3gvbvnjcg95dy6nc0000gn/T/ipykernel_94585/1708691962.py:2: RuntimeWarning: divide by zero encountered in true_divide
  x_user = x_user / x_user.sum(1, keepdims=True)
/var/folders/xh/jth0z5cd3gvbvnjcg95dy6nc0000gn/T/ipykernel_94585/1708691962.py:2: RuntimeWarning: invalid value encountered in true_divide
  x_user = x_user / x_user.sum(1, keepdims=True)


In [5]:
# to tensor
x_user = torch.from_numpy(np.float32(x_user))
x_item = torch.from_numpy(np.float32(x_item))
y_train = torch.from_numpy(np.array(y_train))

num_user_nodes, user_input_dim = x_user.shape
num_item_nodes, item_input_dim = x_item.shape
hidden_dim = 500
embedding_dim = 500
edge_dim = len(x_train_edge['9176-86884'])
num_nodes = num_item_nodes + num_user_nodes

In [6]:
indices = torch.from_numpy(np.array([L.row, L.col]).astype("int64")).long()
values = torch.from_numpy(L.data.astype(np.float32))
adjacency = torch.sparse.FloatTensor(indices, values, L.shape)

In [7]:
adjacency.shape

torch.Size([454244, 454244])

In [8]:
model = Net(user_input_dim, item_input_dim, hidden_dim, embedding_dim, edge_dim, use_edge_feature=False)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()

In [9]:
# train
def train():
    loss_history = []
    acc_history = []
    model.train()
    for epoch in range(num_epochs):

        logits = model(adjacency, x_user, x_item, x_train_edge)
        loss = criterion(logits, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_history.append(loss.item())
        acc = accuracy(logits, y_train)
        acc_history.append(acc.item())
        print("Epoch {:03d}: Loss {:.4f}, TrainAcc {:.4f}".format(
            epoch, loss.item(), acc.item()
        ))
    return loss_history, acc_history

In [10]:
# test
def accuracy(logits, y):
    ones = torch.ones_like(logits)
    zeros = torch.zeros_like(logits)
    y_hat = torch.where(logits > 0.5, ones, logits)
    y_hat = torch.where(logits <= 0.5, zeros, logits)
    return (y_hat == y).sum() / len(y_hat)

# plot
from matplotlib import pyplot as plt
def plot_loss_and_acc(loss_history, acc_history):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.plot(range(len(loss_history)), loss_history,
             c=np.array([255, 71, 90]) / 255.)
    plt.ylabel('Loss')

    ax2 = fig.add_subplot(111, sharex=ax1, frameon=False)
    ax2.plot(range(len(acc_history)), acc_history,
             c=np.array([79, 179, 255]) / 255.)
    ax2.yaxis.tick_right()
    ax2.yaxis.set_label_position("right")
    plt.ylabel('TrainAcc')

    plt.xlabel('Epoch')
    plt.title('Training Loss & Training Accuracy')
    plt.show()

In [11]:
loss, acc = train()
plot_loss_and_acc(loss, acc)

/Users/duran/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: addmm: Argument #3 (dense): Expected dim 0 size 454244, got 450416

In [20]:
num_nodes

450416